In [1]:
from datasets import load_dataset

In [2]:
from llama_index.vector_stores.chroma import ChromaVectorStore

In [7]:
from llama_index.core.indices.vector_store import VectorStoreIndex
from llama_index.core.schema import Document

In [ ]:
import google.protobuf

In [ ]:
print(google.protobuf.__version__)

In [10]:
pwd

'/mnt/f/chatbot_ui/notebooks'

In [12]:
import os
os.getcwd()

'/mnt/f/chatbot_ui/notebooks'

In [14]:
from pathlib import Path

In [17]:
for i in Path(os.getcwd()).parents:
    print(i)

/mnt/f/chatbot_ui
/mnt/f
/mnt
/


In [22]:
DATA_DIR = os.path.join(Path(os.getcwd()).parents[0], 'local_only', 'data')
os.makedirs(DATA_DIR, exist_ok=True)

In [8]:
from datasets import load_dataset

In [24]:
%%time
dataset = load_dataset("rajpurkar/squad_v2", cache_dir = DATA_DIR)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

CPU times: user 1.85 s, sys: 187 ms, total: 2.04 s
Wall time: 6.35 s


In [25]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [29]:
dataset['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [65]:
dataset['train'][1]

{'id': '56be85543aeaaa14008c9065',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'What areas did Beyonce compete in when she was growing up?',
 'answers': {'text': ['singing and dancing'], 'answer_start': [207]}}

In [67]:
import pandas as pd
df = pd.DataFrame(dataset['train'])

In [68]:
df.head()

,id,title,context,question,answers
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start'..."
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,"{'text': ['singing and dancing'], 'answer_star..."
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,"{'text': ['2003'], 'answer_start': [526]}"
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"{'text': ['Houston, Texas'], 'answer_start': [..."
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,"{'text': ['late 1990s'], 'answer_start': [276]}"


In [69]:
df.title.nunique()

442

In [70]:
df.title.unique()

array(['Beyoncé', 'Frédéric_Chopin',
       'Sino-Tibetan_relations_during_the_Ming_dynasty', 'IPod',
       'The_Legend_of_Zelda:_Twilight_Princess', 'Spectre_(2015_film)',
       '2008_Sichuan_earthquake', 'New_York_City',
       'To_Kill_a_Mockingbird', 'Solar_energy', 'Kanye_West', 'Buddhism',
       'American_Idol', 'Dog', '2008_Summer_Olympics_torch_relay',
       'Genome', 'Comprehensive_school', 'Republic_of_the_Congo',
       'Prime_minister', 'Institute_of_technology', 'Wayback_Machine',
       'Dutch_Republic', 'Symbiosis', 'Canadian_Armed_Forces',
       'Cardinal_(Catholicism)', 'Iranian_languages', 'Lighting',
       'Separation_of_powers_under_the_United_States_Constitution',
       'Architecture', 'Human_Development_Index', 'Southern_Europe',
       'BBC_Television', 'Arnold_Schwarzenegger', 'Plymouth', 'Heresy',
       'Warsaw_Pact', 'Materialism', 'Christian',
       'Sony_Music_Entertainment', 'Oklahoma_City', 'Hunter-gatherer',
       'United_Nations_Population_Fund

In [48]:
documents = []
for i in dataset['train'].select(range(500)):
    item = dict(i)
    context = item.pop('context')
    metadata  = item
    documents.append(Document(text=context, metadata=metadata))

In [71]:
documents[1].__dict__

{'id_': 'ad7faea7-0eb5-42a1-9788-ef08b8c6cbd0',
 'embedding': None,
 'metadata': {'id': '56be85543aeaaa14008c9065',
  'title': 'Beyoncé',
  'question': 'What areas did Beyonce compete in when she was growing up?',
  'answers': {'text': ['singing and dancing'], 'answer_start': [207]}},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {},
 'metadata_template': '{key}: {value}',
 'metadata_separator': '\n',
 'text_resource': MediaResource(embeddings=None, data=None, text='Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of B

In [50]:
from llama_index.core import Document, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [51]:
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

In [62]:
vector_store = index.vector_store

In [ ]:

# Check number of nodes stored (works for Chroma, FAISS, etc.)
print(f"Total Nodes Stored: {vector_store._collection.count()}")

In [72]:
from llama_index.core.callbacks import CallbackManager, CBEventType

In [73]:
dir(CBEventType)

['AGENT_STEP',
 'CHUNKING',
 'EMBEDDING',
 'EXCEPTION',
 'FUNCTION_CALL',
 'LLM',
 'NODE_PARSING',
 'QUERY',
 'RERANKING',
 'RETRIEVE',
 'SUB_QUESTION',
 'SYNTHESIZE',
 'TEMPLATING',
 'TREE',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']